In [199]:
import numpy as np

In [55]:
def transpositionMatrix(n,r1,r2):
    """
    This function creates a row permuation matrix P. 

    Input: size n, index of rows r1, r2 to be permuted
    """
    P=np.eye(n); P[r1,r2]=1; P[r2,r1]=1; P[r1,r1]=0; P[r2,r2]=0;
    return P

def forwardSolve(L,B):
    """
    This code implements a forward substitution solver

    Input: Lower diagonal non-singular matrix L, matrix B
    Output: Matrix X such that LX=B
    """
    n=L.shape[0]
    if len(B.shape)<=1:
        m=1
    else:
        m=B.shape[1]

    if B.shape[0]!=n:
        raise ValueError("Incompatible sizes. L has size {} whereas B has size {}.".format(L.shape,B.shape))

    X=np.zeros([n,m])

    for i in range(n):
        if L[i,i]==0:
            raise ValueError("Matrix should be non-singular")
        X[i]=(B[i]-sum([L[i,t]*X[t] for t in range(i)]))/L[i,i]

    return X

def backwardSolve(U,B):
    """
    This code implements a backward substitution solver

    Input: Lower diagonal non-singular matrix L, matrix B
    Output: Matrix X such that LX=B
    """
    n=U.shape[0]
    if len(B.shape)<=1:
        m=1
    else:
        m=B.shape[1]

    if B.shape[0]!=n:
        raise ValueError("Incompatible sizes. L has size {} whereas B has size {}.".format(L.shape,B.shape))

    X=np.zeros([n,m])

    for i in range(n):
        i=n-1-i
        if U[i,i]==0:
            raise ValueError("Matrix should be non-singular")
        X[i]=(B[i]-sum([U[i,t]*X[t] for t in range(i+1,n)]))/U[i,i]

    return X


def LU(A):
    """
    This function implements simple LU decomposition heuristic using Gaussian elimination.
    The complexity of the algorithm is O(n^3/2). WARNING: Not vectorized, so it might be slow for large matrices.
    """
    A=A.astype('f')
    n=A.shape[0]
    L=np.eye(n)
    U=A.copy()
    P=np.eye(n)
    Pinv=np.eye(n)
    for i in range(n):
        stepDone=False
        while  not stepDone: 
            if U[i,i]!=0:
                #Gaussian Elimination Step
                tempL=np.eye(n)
                for i2 in range(i+1,n):
                    l=U[i2,i]/U[i,i]
                    U[i2]=U[i2]-l*U[i]
                    tempL[i2,i]=-l
                L=np.matmul(tempL,L)
                stepDone=True                   
            else:
                #Check non-zero element in column
                i2=i+1
                while i2<n:
                    if U[i2,i]!=0:
                        #Permute rows
                        temp=U[i].copy()
                        U[i]=U[i2]
                        U[i2]=temp
                        #Save Permutation Matrix
                        P=np.matmul(transpositionMatrix(n,i2,i),P)
                        Pinv=np.matmul(Pinv,transpositionMatrix(n,i2,i))
                        break
                    else:
                        i2+=1
                if i2>=n:
                    #No non-zero value found, continue to next step
                    stepDone=True

    L=forwardSolve(L,np.eye(n))    
    return P, Pinv, L, U 

def LUSolver(A,B):
    """
    This program solves the system of linear AX=B equation using LU factorization
    
    Input: Matrices A and B of compatible sizes
    """
    P,Pinv, L,U=LU(A)
    X=forwardSolve(L,np.matmul(Pinv,B))
    X=backwardSolve(U,X)
    
    return X

def Cholesky(A):
    """
    This is a simple algorithm to calculate the cholesky factorization of a
    positive definite matrix. WARNING: Precision is not the best for this one.
    
    Input: Square, positive definite A (no complex for now)
    Output: Lower triangular matrix L and diagonal matrix D such that A=LDL^T
    """
    A=A.astype('f')
    n=A.shape[0]
    
    #Sanity check
    if n!=A.shape[1]:
        raise ValueException("Matrix A should be square matrix")
    if not (A==A.T).all():
        raise ValueException("Matrix A should symmetric")
    
    i=0
    L=np.eye(n)
    while i<n:
        a=A[i,i]
        if a==0:
            raise ValueException("Matrix A is singular.")
        if a<0:
            raise ValueException("Matrix is not positive semidefinite.")
            
        b=A[i+1:,i].reshape((n-i-1,1))
        B=A[i+1:,i+1:]-np.matmul(b,b.T)/a
        
        #Update A
        subA=np.block([[A[:i, :i], np.zeros((A[:i,:i].shape[0],1))],[np.zeros((1,A[:i,:i].shape[1])),a]])
        A=np.block([[subA, np.zeros((subA.shape[0],n-subA.shape[0]))],[np.zeros((n-subA.shape[0],subA.shape[1])), B]])
        
        #Update L
        subL=np.block([[1, np.zeros((1,b.shape[0]))],[b/a,np.eye(b.shape[0]) ]])
        tempL=np.block([[np.eye(i), np.zeros((i,subL.shape[0]))], [np.zeros((subL.shape[0],i)),subL]])
        L=np.matmul(L,tempL)
        i+=1
    return L, A

In [212]:
np.random.seed(0)
Ar=np.random.random(3).reshape((3,1))
Ar=np.eye(3)+np.matmul(r,r.T)
Ar

array([[2.95660408, 1.24660392, 1.05064089],
       [1.24660392, 3.62451883, 1.36914851],
       [1.05064089, 1.36914851, 3.15392176]])

In [213]:
P,Pinv,L,U=LU(Ar)
np.linalg.norm(np.matmul(np.matmul(P,L),U)-Ar)

1.675275486656903e-07

In [216]:
br=np.random.random(3).reshape((3,1))
x=LUSolver(Ar,br)
np.linalg.norm(np.matmul(Ar,x)-br)

2.2373316256180624e-08

In [218]:
x

array([[0.02524883],
       [0.1789237 ],
       [0.08161088]])

In [219]:
np.linalg.solve(Ar,br)

array([[0.02524883],
       [0.1789237 ],
       [0.08161089]])

In [217]:
np.linalg.norm(np.matmul(Ar,np.linalg.solve(Ar,br))-br)

0.0

In [210]:
L,D=Cholesky(r)
np.linalg.norm(np.matmul(np.matmul(L,D),L.T)-r)

6.942356558539425e-08

In [211]:
L2=np.linalg.cholesky(r)
np.linalg.norm(np.matmul(L2,L2.T)-r)

3.1401849173675503e-16

In [200]:
def diag2(x):
    """
    Return diagonal matrix diag(x). Different to np.diag as np.diag of the zero vector returns a 1x1 matrix always.
    """
    n=len(x)
    X=np.zeros((n,n))
    for i in range(n):
        X[i,i]=x[i]
        
    return X
    

def basicInteriorLP(c,A,b):
    """
    This is a basic solver of LP using interior point method algorithm.
    
    Input: Problem in form min c^t x : A x <= b
    Output: Optimal Value, Primal Solution, Dual Solution
    
    WARNING: Not efficient for the moment... Mehrotra's simple implementation in progress
    """
    epsilon=1e-10
    (m,n)=A.shape
    
    x=np.ones((n,1))
    s=np.ones((n,1))
    l=np.ones((m,1))
    
    #duality measure
    mu=np.mean(x*s)
    err1=np.linalg.norm(b-np.matmul(A,x))/max([np.linalg.norm(b),1])
    err2=np.linalg.norm(c-s-np.matmul(A.T,l))/max([np.linalg.norm(c),1])

    while max([mu,err1,err2])>epsilon:
        err1=np.linalg.norm(b-np.matmul(A,x))/max([np.linalg.norm(b),1])
        err2=np.linalg.norm(c-s-np.matmul(A.T,l))/max([np.linalg.norm(c),1])
        
        print("Central: {} Primal: {} Dual: {}".format(mu,err1,err2))
        #Compute the jacobian and function:
        J=np.block([[np.zeros((n,n)),A.T,np.eye(n)],[A,np.zeros((m,m)),np.zeros((m,n))],[diag2(s),np.zeros((n,m)), diag2(x)]])
        F=np.block([[np.matmul(A.T,l)+s-c],[np.matmul(A,x)-b],[x*s]])
        
        #Solve the Newton step:
        
        #Find an LU factorization of the jacobian
        P,Pinv,L,U=LU(J)
        
        #Find the affine scaling direction:
        aff=forwardSolve(L,np.matmul(Pinv,-F))
        aff=backwardSolve(U,aff)
        
        aff=np.linalg.solve(J,-F)
        
        #Determine if step is good
        dx=aff[:n]
        ds=aff[-n:]
        dl=aff[n:-n]
        delta1=min([1]+[-x[i]/dx[i] for i in range(n) if dx[i]<0])
        delta2=min([1]+[-s[i]/ds[i] for i in range(n) if ds[i]<0])
        
        if min([delta1,delta2])<1:
            #not full step possible, we need center-correction
            
            muaff=np.mean((x+delta1*dx)*(s+delta2*ds))
            sigma=min(0.208, (muaff/mu)**2)
            
            F2=np.block([[np.zeros((n+m,1))],[aff[:n]*aff[-n:]-sigma*muaff*np.ones((n,1))]])
            
            #ccorr=forwardSolve(L,np.matmul(Pinv,-F2))
            #ccorr=backwardSolve(U,ccorr)
            
            ccorr=np.linalg.solve(J,-F2)
            
            dx+=ccorr[:n]
            dl+=ccorr[n:-n]
            ds+=ccorr[-n:]
        
        #step length to guarantee feasibility
        delta1=min([1]+[-x[i]/dx[i] for i in range(n) if dx[i]<0])
        delta2=min([1]+[-s[i]/ds[i] for i in range(n) if ds[i]<0])
        
        #step length to guarantee that we are close to the central path:
        
        for alpha in [1, .9975, .95, .90, .75, .50]:
            xs=(x+alpha*delta1*dx)*(s+alpha*delta2*ds)
            if np.min(xs)>=1e-6*np.mean(xs):
                delta1=alpha*delta1
                delta2=alpha*delta2
                break
        
        x+=delta1*dx
        s+=delta2*ds
        l+=delta2*dl
        
        mu=np.mean(x*s)
    return np.matmul(c.T,x), x,s  

In [201]:
A=np.array([[1,4,1,0],[1,1,0,1]])
b=np.array([[8],[4]])
c=np.array([[2],[5],[0],[0]])

In [202]:
basicInteriorLP(-c,A,b)

Central: 1.0 Primal: 0.25 Dual: 2.3044185443591205
Central: 0.2475916448278927 Primal: 0.03850594572783072 Dual: 0.35202986224744426
Central: 0.01897211394479829 Primal: 9.626486431957293e-05 Dual: 0.015113703207410645
Central: 4.967101781870736e-05 Primal: 2.4917644372290935e-07 Dual: 3.7784258018603556e-05
Central: 1.2417928376690224e-07 Primal: 6.229414539760129e-10 Dual: 9.446064510742916e-08
Central: 3.1044822002444885e-10 Primal: 1.557309836641707e-12 Dual: 2.3615156258039155e-10
Central: 7.761205501273799e-13 Primal: 3.9968028886505635e-15 Dual: 5.903733510635419e-13


(array([[-12.]]),
 array([[2.66666667e+00],
        [1.33333333e+00],
        [2.74068080e-16],
        [6.46955452e-15]]),
 array([[3.09190749e-16],
        [1.44805677e-16],
        [1.00000000e+00],
        [1.00000000e+00]]))